In [5]:
import os
import shutil
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import keras

In [6]:
# 1) Load & merge the data
# Path to your goemotions folder
data_path = "/Users/aaronfischer/Desktop/Catolica WS 24/Trimester 4/Advanced Topics in Predictive Analytics/Project/Project Files-20250428/goemotions/data/full_dataset"  # ← Change this to your actual path

# List all CSV files
csv_files = [f for f in os.listdir(data_path) if f.endswith(".csv")]

# Read and concatenate
dfs = [pd.read_csv(os.path.join(data_path, f)) for f in csv_files]
goemo_df = pd.concat(dfs, ignore_index=True)

print(goemo_df.head())
print(goemo_df.columns)

# 2) Filter out “very unclear” examples
#    (use bracket notation so you don’t get AttributeError)
if 'example_very_unclear' in goemo_df.columns:
    df = goemo_df[goemo_df['example_very_unclear'] == False].copy()
else:
    # if your column has a different name, replace 'example_very_unclear' below
    df = goemo_df.copy()

                                                text       id  \
0                                    That game hurt.  eew5j0j   
1   >sexuality shouldn’t be a grouping category I...  eemcysk   
2     You do right, if you don't care then fuck 'em!  ed2mah1   
3                                 Man I love reddit.  eeibobj   
4  [NAME] was nowhere near them, he was by the Fa...  eda6yn6   

                author            subreddit    link_id   parent_id  \
0                Brdd9                  nrl  t3_ajis4z  t1_eew18eq   
1          TheGreen888     unpopularopinion  t3_ai4q37   t3_ai4q37   
2             Labalool          confessions  t3_abru74  t1_ed2m7g7   
3        MrsRobertshaw             facepalm  t3_ahulml   t3_ahulml   
4  American_Fascist713  starwarsspeculation  t3_ackt2f  t1_eda65q2   

    created_utc  rater_id  example_very_unclear  admiration  ...  love  \
0  1.548381e+09         1                 False           0  ...     0   
1  1.548084e+09        37               

In [13]:
from transformers import AutoTokenizer

# initialize your tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=True)

# your text column
text_col = "text"

# 1) Simple (Python loop)
max_len = max(
    len(tokenizer(txt, truncation=False)["input_ids"])
    for txt in df[text_col]
)
print(f"Max tokens = {max_len}")

# 2) Batched (faster for large data)
enc = tokenizer(
    df[text_col].tolist(),
    truncation=False,
    return_length=True,
    add_special_tokens=True
)
max_len = max(enc["length"])
print(f"Max tokens = {max_len}")

Max tokens = 316
Max tokens = 316


In [4]:
def load_imdb_data(data_dir):
    data = {"review": [], "sentiment": []}
    
    for split in ["train", "test"]:
        for sentiment in ["pos", "neg"]:
            sentiment_path = os.path.join(data_dir, split, sentiment)
            for file_name in os.listdir(sentiment_path):
                if file_name.endswith(".txt"):
                    file_path = os.path.join(sentiment_path, file_name)
                    with open(file_path, encoding="utf-8") as f:
                        review = f.read()
                        data["review"].append(review)
                        data["sentiment"].append(1 if sentiment == "pos" else 0)
    
    return pd.DataFrame(data)

# Usage
data_dir = "/Users/aaronfischer/Desktop/Catolica WS 24/Trimester 4/Advanced Topics in Predictive Analytics/Project/Project Files-20250428/aclImdb"
imdb_df = load_imdb_data(data_dir)

print(imdb_df.head())

                                              review  sentiment
0  For a movie that gets no respect there sure ar...          1
1  Bizarre horror movie filled with famous faces ...          1
2  A solid, if unremarkable film. Matthau, as Ein...          1
3  It's a strange feeling to sit alone in a theat...          1
4  You probably all already know this by now, but...          1


In [16]:
import numpy as np
from transformers import AutoTokenizer

# initialize your tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=True)

# 1) Compute lengths for each review
lengths = imdb_df["review"].apply(
    lambda txt: len(tokenizer(txt, truncation=False)["input_ids"])
)

# total tokens across all reviews
total_tokens = lengths.sum()

# 1b) Count how many exceed various thresholds, and token‐loss proportion
thresholds = [300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300]
for t in thresholds:
    mask = lengths > t
    cnt_reviews = mask.sum()
    lost_tokens = lengths[mask].sum()
    prop_tokens = lost_tokens / total_tokens

    print(f"> {t:4d} tokens:  {cnt_reviews:6d} reviews  |  "
          f"token-loss = {lost_tokens:8d} ({prop_tokens:.2%})")

# 2) Find the index of the longest one
max_idx = lengths.idxmax()
max_len = lengths.loc[max_idx]

# 3) Grab the review text at that index
longest_review = imdb_df.loc[max_idx, "review"]

# 4) Print everything out
print(f"\nIndex of longest review: {max_idx}")
print(f"Max tokens = {max_len}")
print("---- Review text ----")
print(longest_review)


Token indices sequence length is longer than the specified maximum sequence length for this model (670 > 512). Running this sequence through the model will result in indexing errors


>  300 tokens:   17591 reviews  |  token-loss =  9576183 (61.72%)
>  400 tokens:   11371 reviews  |  token-loss =  7424839 (47.85%)
>  500 tokens:    7587 reviews  |  token-loss =  5735694 (36.97%)
>  600 tokens:    5159 reviews  |  token-loss =  4409980 (28.42%)
>  700 tokens:    3547 reviews  |  token-loss =  3367894 (21.71%)
>  800 tokens:    2456 reviews  |  token-loss =  2553080 (16.45%)
>  900 tokens:    1726 reviews  |  token-loss =  1935186 (12.47%)
> 1000 tokens:    1180 reviews  |  token-loss =  1418235 (9.14%)
> 1100 tokens:     800 reviews  |  token-loss =  1020356 (6.58%)
> 1200 tokens:     497 reviews  |  token-loss =   672205 (4.33%)
> 1300 tokens:     262 reviews  |  token-loss =   379070 (2.44%)

Index of longest review: 30508
Max tokens = 3157
---- Review text ----
There's a sign on The Lost Highway that says:<br /><br />*MAJOR SPOILERS AHEAD*<br /><br />(but you already knew that, didn't you?)<br /><br />Since there's a great deal of people that apparently did not ge